# Clean Corpus

## In this notebook, I will clean and sort my data

In [2]:
import os
# importing required modules
import PyPDF2
  
import matplotlib.pyplot as plt
from collections import Counter
import json
import pickle

In [8]:
os.chdir('/Commjhub/jupyterhub/comm318_fall2019/sstrickberger/comm313_S21/comm313_S21_Final_Project')

In [9]:
os.getcwd()

'/Commjhub/jupyterhub/comm318_fall2019/sstrickberger/comm313_S21/comm313_S21_Final_Project'

In [11]:
%run data_analysis/functions.ipynb

### Let's get a sense of what is in my data

In [6]:
print("This is a name of my data so far: ", os.listdir('data'))

This is a name of my data so far:  ['Abraham D. Shadd_1840.pdf', 'Amos G. Beman_1847-10-06.pdf', 'Andrew Harris_1839-05-16.pdf', 'Anonymous_1831-09_999.pdf', "Bob'n Around_1860.pdf", 'Charles Bennet Ray_1849-04-22.pdf', 'Charles Bennet Ray_1850-10-05.pdf', 'Charles Henry Langston_1859-12-02.pdf', 'Charles L. Reason_1849-04-22.pdf', 'Charles L. Reason_1853-03-04.pdf', 'Charles Lenox Remond_1848_281.pdf', 'Charles Lenox Remond_1855_457.pdf', 'Charles Lenox Remond_1856-05-28.pdf', 'Charles Lenox Remond_1858-03-08.pdf', 'Charles V. Caples_1835-10-05.pdf', 'Ezra R. Johnson_1862-09-03.pdf', 'Frances Ellen Watkins Harper_1858-05-11.pdf', 'H. Ford Douglass_1860_353.pdf', 'Henry Bibb_1847-08-11.pdf', 'Henry Bibb_1850-04-02.pdf', 'Henry Highland Garnet_1859.pdf', 'Henry Highland Garnet_1859_510.pdf', 'Henry Highland Garnet_1861-10-23.pdf', 'Henry Highland Garnet_1862-05-12.pdf', 'Henry Highland Garnet_1863-04-20_137.pdf', 'J. B. Smith_1852.pdf', 'J. W. C. Pennington_1849-12.pdf', 'J. W. C. Penni

In [7]:
os.listdir('data')

['Abraham D. Shadd_1840.pdf',
 'Amos G. Beman_1847-10-06.pdf',
 'Andrew Harris_1839-05-16.pdf',
 'Anonymous_1831-09_999.pdf',
 "Bob'n Around_1860.pdf",
 'Charles Bennet Ray_1849-04-22.pdf',
 'Charles Bennet Ray_1850-10-05.pdf',
 'Charles Henry Langston_1859-12-02.pdf',
 'Charles L. Reason_1849-04-22.pdf',
 'Charles L. Reason_1853-03-04.pdf',
 'Charles Lenox Remond_1848_281.pdf',
 'Charles Lenox Remond_1855_457.pdf',
 'Charles Lenox Remond_1856-05-28.pdf',
 'Charles Lenox Remond_1858-03-08.pdf',
 'Charles V. Caples_1835-10-05.pdf',
 'Ezra R. Johnson_1862-09-03.pdf',
 'Frances Ellen Watkins Harper_1858-05-11.pdf',
 'H. Ford Douglass_1860_353.pdf',
 'Henry Bibb_1847-08-11.pdf',
 'Henry Bibb_1850-04-02.pdf',
 'Henry Highland Garnet_1859.pdf',
 'Henry Highland Garnet_1859_510.pdf',
 'Henry Highland Garnet_1861-10-23.pdf',
 'Henry Highland Garnet_1862-05-12.pdf',
 'Henry Highland Garnet_1863-04-20_137.pdf',
 'J. B. Smith_1852.pdf',
 'J. W. C. Pennington_1849-12.pdf',
 'J. W. C. Pennington_18

## Sort data into two distinct corpora: pre-1840 and post-1840

### Organize the meta data

In [8]:
#only sift through pdf files
filenames = [name for name in os.listdir("./data/") if name.endswith(".pdf")]

In [9]:
len(filenames)

735

In [10]:
#load in the json file
f = open('data/metadata.json',)
  
# returns JSON object as 
# a dictionary
meta_data = json.load(f)

In [11]:
clean_md_proto = []
for item in meta_data:
    if item != None :
        clean_md_proto.append(item)
        
len(clean_md_proto) #good this matches the number of file names

734

In [12]:
clean_md = []
for md in clean_md_proto:
    if md['Data_note'] == 'Complete':
        clean_md.append(md)

len(clean_md) #we dropped one

733

### What years are represented in this data?

In [13]:
#better understand the data

for md in clean_md:
    #print(md['Data_note'])
    #if md['Data_note'] == "Complete":
    year = md['Date published'].split("-")[0]

    md["year"] = int(year)


In [14]:
years = []
for md in clean_md:
    if md['year'] == "Unknown":
        continue
    years.append(md['year'])

In [15]:
#what years are being represented?

year_counter = {i:years.count(i) for i in years}

year_counter

{1830: 1,
 1831: 10,
 1832: 2,
 1833: 4,
 1834: 11,
 1835: 3,
 1836: 16,
 1837: 17,
 1838: 17,
 1839: 11,
 1840: 14,
 1841: 10,
 1842: 6,
 1843: 15,
 1844: 9,
 1845: 2,
 1846: 4,
 1847: 10,
 1848: 9,
 1849: 36,
 1850: 37,
 1851: 31,
 1852: 20,
 1853: 26,
 1854: 24,
 1855: 40,
 1856: 12,
 1857: 21,
 1858: 15,
 1859: 66,
 1860: 35,
 1861: 56,
 1862: 30,
 1863: 59,
 1864: 20,
 1865: 34}

In [16]:
#how many files before 1840 (inclusive)
number_before_1840 = 0

for year in range(1830,1841):
    number_before_1840 = year_counter[year] + number_before_1840
    
number_before_1840

106

In [17]:
#how many files after 1840 (exclusive)
number_after_1840 = 0

for year in range(1841,1866):
    number_after_1840 = year_counter[year] + number_after_1840
    
number_after_1840

627

In [18]:
pre_1840 = []
post_1840 = []
for md in clean_md:
    #print(md['year'])
    
    if md['year'] < 1841:
        pre_1840.append(md['filename'])
    if md['year'] > 1840:
        post_1840.append(md['filename'])
        
        
print(pre_1840[:5])
print(post_1840[:5])

['Solomon R. Alexander_1840-08-03.pdf', '"A Colored Female"_1834.pdf', 'Anonymous_1831-08-05.pdf', 'Anonymous_1831-09.pdf', 'Augustus William Hanson_1838-09-03.pdf']
['S. M. Africanus_1849-09-13_979.pdf', 'William G. Allen_1853-12-01.pdf', 'William G. Allen_1853.pdf', 'William G. Allen_1855.pdf', 'William G. Allen_1852-06-22.pdf']


In [19]:
print(len(pre_1840)) 
print(len(post_1840))
#as with above, we have 106 sermons from 1840 and before, and 627 from 1840 and after

106
627


In [20]:
#TRY TO MAKE A FUNcTINO??
#def document_compiler(name):
  #3  """
   # We want to output organized set of documents, 
   # by e.g. year or publication type
    
   # argument: document_name 
   # values: clean_pages - a compiled set of pages
   # """
   

## Clean all pages

* I want to remove excess text on each page, such as document labels that the archive added to them
* To to so, I will run some for-loops and remove this unwanted text.

### Clean pre-1840 pages

In [21]:
# creating a pdf file object
clean_pages = []
for name in pre_1840:
    
    pdfFileObj = open(os.path.join('data', name), 'rb')
  
    #creating a pdf reader object
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
  
    # printing number of pages in pdf file
    # print(pdfReader.numPages)
  
    #this is to loop through page numbers of the document
    for Y in range(0,6): 
        #this is to loop through line numbers, looking for where to cut the document off
        for X in range(2,12): 
      
    
        
            try:
                # creating a page object
                pageObj = pdfReader.getPage(Y)
  
                # extracting text from page
                page = pageObj.extractText()
    
                page_strip = page.strip().split('\n')


                if (page_strip[X].strip().endswith(str(Y+1))):
                    clean_page = page.strip().split('\n')[X+1:len(page_strip)]
                    
                    clean_pages.append(clean_page)
            except: 
                break

print(clean_pages)    
          
                
    #MAKE SURE TO CHECK THE DATA AT END with the PDF

  
    # closing the pdf file object
    # pdfFileObj.close()

[['    Mr. S. R. ALEXANDER then addressed the assembly.  ', 'Friends and Fellow citizens, he said, I am well  ', '', 'aware that it requires a better head than I can pretend  ', '', 'to, to do justice to this great theme, but I am encour- ', '', 'aged, enlightened and cheered as I look around me.   ', '', 'I confess my incompetence in one point of view, but  ', '', 'I know you will all respond to what I feel.  I can  ', '', 'express to you the feelings of ', 'one heart in regard  ', 'to liberty.  The cheering occasion enlivens every  ', 'one of us.  To-day we rejoice with those who re- ', '', 'joice in their freedom across the water, yet is our joy  ', '', 'mixed with sorrow.  They are free in the West Indies,  ', '', 'but our brethren in this land are in slavery still.  I  ', '', 'trust the time is not far distant when we shall meet  ', '', 'to celebrate their emancipation also, in such numbers  ', '', 'as to need a larger place than this, in which to hold  ', '', 'the celebration.  B

In [22]:
#How many pages of writing before 1840, inclusive?
len(clean_pages)
clean_pages_pre_1840 = clean_pages

In [23]:
len(pre_1840)

106

In [24]:
len(post_1840)

627

In [25]:
for name in post_1840[:3]:
    print(name)

S. M. Africanus_1849-09-13_979.pdf
William G. Allen_1853-12-01.pdf
William G. Allen_1853.pdf


### Clean post-1840 pages

In [26]:
clean_pages_post_1480 = []
for name in post_1840:
    
    print(name)
    pdfFileObj = open(os.path.join('data', name), 'rb')

  
    #creating a pdf reader object
    try:
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

        # printing number of pages in pdf file
        # print(pdfReader.numPages)

        #this is to loop through page numbers of the document
        for Y in range(0,6): 
            #this is to loop through line numbers, looking for where to cut the document off
            for X in range(2,12): 



                try:
                    # creating a page object
                    pageObj = pdfReader.getPage(Y)

                    # extracting text from page
                    page = pageObj.extractText()

                    page_strip = page.strip().split('\n')


                    if (page_strip[X].strip().endswith(str(Y+1))):
                        clean_page = page.strip().split('\n')[X+1:len(page_strip)]

                        clean_pages_post_1480.append(clean_page)
                except: 
                    break
    except: 
        print("didn't work for", name)

S. M. Africanus_1849-09-13_979.pdf


William G. Allen_1853-12-01.pdf
William G. Allen_1853.pdf
William G. Allen_1855.pdf
William G. Allen_1852-06-22.pdf
William G. Allen_1853_479.pdf
William G. Allen_1855_923.pdf
William G. Allen_1854.pdf
William Johnson Alston_1861-09-26.pdf
John Anderson_1863.pdf
John Anderson_1863_187.pdf
John Anderson_1861-06-19.pdf
Frederick G. Barbadoes_1856-10-25.pdf
Ebenezer D. Bassett_1861.pdf
Ebenezer D. Bassett_1862-08-25.pdf
Philip A. Bell_1862.pdf
Phillip A. Bell_1863-09-15.pdf
Amos G. Beman_1841.pdf
Amos G. Beman_1847-10-06.pdf
Jehiel C. Beman_1843-08-01.pdf
Henry Bibb_1849-04-22.pdf
Henry Bibb_1847-08-11.pdf
Henry Bibb_1850-04-02.pdf
Henry Bibb_1848-07-06.pdf
Henry Bibb_1844.pdf
Henry Bibb_1849.pdf
Edward Wilmot Blyden_1862-06-26.pdf
Edward Wilmot Blyden_1862.pdf
Edward Wilmot Blyden_1857-07-27.pdf
Edward Wilmot Blyden_1862_437.pdf
Edward Wilmot Blyden_1865-07-26.pdf
Edward Wilmot Blyden_1861-07-21.pdf
Edward Wilmot Blyden_1862_658.pdf
Edward Wilmot Blyden_1862-06.pdf
Edward Wilmot Blyden_1

Ezra R. Johnson_1863-07-04.pdf
Henry W. Johnson_1852-04-01.pdf
Henry W. Johnson_1848-08-01.pdf
William F. Johnson_1852-03-30.pdf
William H. Johnson_1859-07-04.pdf
William P. Johnson_1843.pdf
Josiah Jones_1842-08-06.pdf
Thomas H. Jones_1850.pdf
William Jones_1861-09-19.pdf
William Jones_1846.pdf
Edmund Kelly_1853.pdf
Edumund Kelly_1853.pdf
Thomas M. Kinnaird_1861.pdf
Thomas M. Kinnard_1861.pdf
Lunsford Lane_1855.pdf
Lunsford Lane_1842-05-11.pdf
Charles Henry Langston_1857-01-21.pdf
Charles Henry Langston_1859-05-12.pdf
Charles Henry Langston_1859-12-02.pdf
Charles Henry Langston_1851-01-15.pdf
John Mercer Langston_1865-02-28.pdf
John Mercer Langston_1859-09-04.pdf
John Mercer Langston_1853.pdf
John Mercer Langston_1865-10-25.pdf
John Mercer Langston_1858-08-02.pdf
John Mercer Langston_1865-03-05.pdf
John Mercer Langston_1855-05-09.pdf
John Mercer Langston_1865.pdf
John Mercer Langston_1865-01-02.pdf
John Mercer Langston_1851-01-15.pdf
John Mercer Langston_1858.pdf
John Mercer Langston_1

Edward Scott_1857.pdf
Nicholas N. Selby_1847.pdf
Wallace Shelton_1849.pdf
Wallace Shelton_1844-08-01.pdf
John W. Simpson_1860.pdf
J. B. Smith_1852.pdf
John B. Smith_1863-01-01.pdf
John B. Smith_1861.pdf
John B. Smith_1859.pdf
John B. Smith_1860-01-16.pdf
James McCune Smith_1859-12-19.pdf
James McCune Smith_1855.pdf
James McCune Smith_1855_252.pdf
James McCune Smith_1841-01.pdf
James McCune Smith_1852-01-13.pdf
James McCune Smith_1841-02-26.pdf
James McCune Smith_1850-10-01.pdf
James McCune Smith_1849.pdf
Lewis Smith_1860.pdf
N. Smith_1844-05-27.pdf
Samuel Snowden_1843-08-01.pdf
William Spellman_1849-09-13.pdf
Benjamin Stanley_1850-10-01.pdf
Austin Steward_1863.pdf
Austin Stewart_1841-08-25.pdf
Rev. Mr. Stewart_1860.pdf
William Still_1860-08-04.pdf
William Still_1860.pdf
William Still_1860-03-02.pdf
D. P. Stokes_1855-11-21.pdf
Thomas H. Street_1864-01-01.pdf
Emeline Sullivan_1861-01-21.pdf
Charles Tasco_1863.pdf
Theodore S. Wright Titus_1859-12-19.pdf
William H. Topp_1852-09-30.pdf
Sojo

In [27]:
# how many pages are there?
len(clean_pages_post_1480)

2089

In [28]:
#check out a  page
clean_pages_post_1480[1]

['  niches of the walls of the houses of their relatives ',
 'where they were seen.  So',
 ' in New York State, the colored citizen was not wholly lost sight of. ',
 '   If he were worth the sum of 325 dollars in real- ',
 '',
 'estate, he could vote, but not otherwiseŠin short, ',
 '',
 'his rights were embalmed, and placed in the niches ',
 'of the political temple, where they could be seen and ',
 'spoken of.  But they had no vitalityŠthey were ',
 '',
 'merely the rights of sepulchre; but even these were ',
 '',
 'not tolerated in Connecticut.  But he believed they ',
 '',
 'would be in time, if colored men would show those ',
 'who truly love republican principles that they could ',
 'appreciate a freeman™s privileges and exercise a free- ',
 '',
 'man™s franchise with reference to the best interests ',
 '',
 'of our common country.']

## Export my two cleaned data sets

In [43]:
#save to file: pre 1840 paages
#file_Name = "data_analysis/clean_pages_pre_1840"
file_Name = "data/clean/clean_pages_pre_1840"
# open the file for writing
fileObject = open(file_Name,'wb') 

# this writes the object a to the
# file named 'testfile'
pickle.dump(clean_pages_pre_1840,fileObject)   

# here we close the fileObject
fileObject.close()

In [45]:
#save to file: post 1840 paages
#file_Name = "data_analysis/clean_pages_post_1840"
file_Name = "data/clean/clean_pages_post_1840"
# open the file for writing
fileObject = open(file_Name,'wb') 

# this writes the object a to the
# file named 'testfile'
pickle.dump(clean_pages_post_1480,fileObject)   

# here we close the fileObject
fileObject.close()